<a href="https://colab.research.google.com/github/jaykradadiya/SummerInternship2022/blob/main/MLJar%2Cpytorch/EvolutionaryNN_TPOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def encode_text_features(encode_decode, data_frame, encoder_isa=None, encoder_mem_type=None):
  # Implement Categorical OneHot encoding for ISA and mem-type
  if encode_decode == 'encode':
    encoder_isa = ce.one_hot.OneHotEncoder(cols=['isa'])
    encoder_mem_type = ce.one_hot.OneHotEncoder(cols=['mem_type'])
    encoder_isa.fit(data_frame, verbose=1)
    df_new1 = encoder_isa.transform(data_frame)
    encoder_mem_type.fit(df_new1, verbose=1)
    df_new = encoder_mem_type.transform(df_new1)
    encoded_data_frame = df_new
  else:
    df_new1 = encoder_isa.transform(data_frame)
    df_new = encoder_mem_type.transform(df_new1)
    encoded_data_frame = df_new
        
  return encoded_data_frame, encoder_isa, encoder_mem_type

In [ ]:
def split_dataset(X, y):
  # split into train and test
  #X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(X, y, X.index, test_size=0.2, random_state=42)
  # use default random state of np.random, so not providing any value here
  X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(X, y, X.index, test_size=0.2, random_state=0)
  return X_train, X_test, y_train, y_test, indices_train, indices_test

In [ ]:
def scale_train_dataset(X_train, y_train, input_scaler=None, output_scaler=None):
  # scale X_train input (fit and transform)
  if input_scaler is not None:
    # fit scaler on training dataset
    input_scaler.fit(X_train)
    # transform training dataset
    X_train = input_scaler.transform(X_train)        
        
  # scale y_train input (fit and transform)
  if output_scaler is not None:
    # fit scaler on training dataset
    output_scaler.fit(y_train)
    # transform training dataset
    y_train = output_scaler.transform(y_train)        
        
  return X_train, y_train, input_scaler, output_scaler

In [ ]:
def scale_test_dataset(X_test, y_test, input_scaler, output_scaler):
  # scale X_test input (only transform)
  if input_scaler is not None:
    # transform test dataset
    X_test = input_scaler.transform(X_test)
        
  # scale y_train input (only transform)
  if output_scaler is not None:
    # transform test dataset
    y_test = output_scaler.transform(y_test)
        
  return X_test, y_test

In [ ]:
def perform_dataset_ops(benchmark, algo, algo_fname):
  print(benchmark, algo, algo_fname)
  df_orig=pd.read_csv(drive_location+algo_fname+".csv")
  #df_orig.dropna(inplace=True)
    
  # Encode text features
  df_encoded, encoder_isa, encoder_mem_type = encode_text_features('encode', df_orig)
  print("df_orig====================>",df_orig)
  print("df_encoded====================>",df_encoded)
  # store learned models in dictory
  dataset = {}
  dataset['encoder_isa'] = encoder_isa
  dataset['encoder_mem_type'] = encoder_mem_type
    
  dataset['df_orig'] = df_orig
  dataset['df_encoded'] = df_encoded

#  for ps_indx, ps in enumerate(df_orig.PS.unique()):
#    print(ps_indx, ps)
#    dataset[ps] = {}
    
#    criteria_df = (df_encoded['PS'] == ps)
    #print(criteria_df)
#    df_selected = df_encoded[criteria_df]
  df_selected = df_encoded    
  if (benchmark == "SPEC2006" or benchmark == "SPEC2017"):
    X_all=df_selected.drop(columns=['arch','ld_shared_by_cores','l2_shared_by_cores','no_of_threads','system_name','bus_speed_qpi','bus_speed_dmi','l1_ins_assoc','l1_data_assoc','l2_assoc','l3_assoc','raw_bus_speed','converted_bus_speed','ddr_type','runtime'])
  elif (benchmark == "NPB"):
    X_all=df_selected.drop(columns=['sys','sysname','arch','l1d_assoc','l1d_cache_lines','l1d_shared_by_threads','l2_assoc','l2_cache_lines','l2_shared_by_threads','l3_assoc','l3_cache_lines','l3_shared_by_threads','PS','runtime','executable','system'])
  else:
    X_all=df_selected.drop(columns=['sys','arch','l1d_assoc','l1d_cache_lines','l1d_shared_by_threads','l2_assoc','l2_cache_lines','l2_shared_by_threads','l3_assoc','l3_cache_lines','l3_shared_by_threads','runtime'])
  #print(X_all)
  X_all.dropna(inplace=True)
  dataset['X_all'] = X_all
  
  # Output
  y=df_selected[['runtime']]
  print("y shape ",y.shape, "y types", y.dtypes)
  #print("y shape ",y.shape, "y types", y.dtypes)
  dataset['y'] = y

  # split into train and test
  X_train, X_test, y_train, y_test, indices_train, indices_test = split_dataset(X_all, y)
  dataset['X_train'] = X_train
  dataset['X_test'] = X_test
  dataset['y_train'] = y_train
  dataset['y_test'] = y_test
  dataset['indices_train'] = indices_train
  dataset['indices_test'] = indices_test

  # unscaled inputs
  # scale train data (fit and transform)
  X_train_unscaled, y_train_unscaled, input_scaler_unscaled, output_scaler_unscaled = scale_train_dataset(X_train, y_train, None, StandardScaler())
  # scale test data (only transform)
  X_test_unscaled, y_test_unscaled = scale_test_dataset(X_test, y_test, input_scaler_unscaled, output_scaler_unscaled)
  dataset['X_train_unscaled'] = X_train_unscaled
  dataset['X_test_unscaled'] = X_test_unscaled
  dataset['y_train_unscaled'] = y_train_unscaled
  dataset['y_test_unscaled'] = y_test_unscaled
  dataset['input_scaler_unscaled'] = input_scaler_unscaled
  dataset['output_scaler_unscaled'] = output_scaler_unscaled
      
  #standardized inputs
  # scale train data (fit and transform)
  X_train_standardized, y_train_standardized, input_scaler_standardized, output_scaler_standardized = scale_train_dataset(X_train, y_train, StandardScaler(), StandardScaler())
  # scale test data (only transform)
  X_test_standardized, y_test_standardized = scale_test_dataset(X_test, y_test, input_scaler_standardized, output_scaler_standardized)
  dataset['X_train_standardized'] = X_train_standardized
  dataset['X_test_standardized'] = X_test_standardized
  dataset['y_train_standardized'] = y_train_standardized
  dataset['y_test_standardized'] = y_test_standardized
  dataset['input_scaler_standardized'] = input_scaler_standardized
  dataset['output_scaler_standardized'] = output_scaler_standardized
      
  save_learned_models(dataset, algo, "dataset")

In [ ]:
def save_learned_models(dct, algo, fname):
  pickleFile = drive_location+algo+"_"+fname+".pickle"
  with open(pickleFile, 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(dct, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
def load_learned_models(algo, fname):
  pickleFile = drive_location+algo+"_"+fname+".pickle"
  f = open(pickleFile, 'rb')
  # disable garbage collector
  gc.disable()
  # The protocol version used is detected automatically, so we do not
  # have to specify it.
  data = pickle.load(f)
  # enable garbage collector again
  gc.enable()
  f.close()
  return data

In [ ]:
def build_nn_model(algo):
  dataset = load_learned_models(algo, "dataset")
  nnmodel = {}
  input_dim = dataset['X_train_standardized'].shape[1]
  output_dim = dataset['y_train_standardized'].shape[1]
  print('input_dim:', input_dim, 'output_dim:', output_dim)
  model_iter = 0
  rmse = 9999999999999999999999999
  model = tf.keras.Sequential()
  model.add(tf.keras.Input(shape=(input_dim,)))
  while (True):
    if (model_iter > 0):
      # add layer with neurons = rmse*100
      model.add(tf.keras.layers.Dense(int(rmse*100), activation=tf.nn.relu))
    model.add(tf.keras.layers.Dense(output_dim, activation='linear'))
    # compile model
    model.compile(loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()], optimizer='adam')
    # fit model
    model.fit(dataset['X_train_standardized'], dataset['y_train_standardized'], epochs=100, verbose=0)
    # evaluate the model
    y_train_pred = model.predict(dataset['X_train_standardized'])
    y_test_pred = model.predict(dataset['X_test_standardized'])
    prev_rmse = rmse
    rmse = mean_squared_error(dataset['y_train_standardized'],y_train_pred,squared=False)
    print(rmse, prev_rmse)
    if (rmse > prev_rmse):
      break
    #nnmodel['model'] = model
    model.save(drive_location+algo+"_nnmodel.h5")
    nnmodel[model_iter] = {}
    nnmodel[model_iter]['y_train'] = dataset['y_train_standardized']
    nnmodel[model_iter]['y_train_pred'] = y_train_pred
    nnmodel[model_iter]['y_test'] = dataset['y_test_standardized']
    nnmodel[model_iter]['y_test_pred'] = y_test_pred
    nnmodel[model_iter]['train_rmse'] = mean_squared_error(dataset['y_train_standardized'],y_train_pred,squared=False)
    nnmodel[model_iter]['train_medae'] = median_absolute_error(dataset['y_train_standardized'],y_train_pred)
    nnmodel[model_iter]['train_explained_var'] = explained_variance_score(dataset['y_train_standardized'],y_train_pred)
    nnmodel[model_iter]['test_rmse'] = mean_squared_error(dataset['y_test_standardized'],y_test_pred,squared=False)
    nnmodel[model_iter]['test_medae'] = median_absolute_error(dataset['y_test_standardized'],y_test_pred)
    nnmodel[model_iter]['test_explained_var'] = explained_variance_score(dataset['y_test_standardized'],y_test_pred)
    nnmodel[model_iter]['y_train_pred_error'] = np.abs(dataset['y_train_standardized'] - y_train_pred/dataset['y_train_standardized']*100)
    nnmodel[model_iter]['y_test_pred_error'] = np.abs(dataset['y_test_standardized'] - y_test_pred /dataset['y_test_standardized']*100)
    nnmodel[model_iter]['train_medape'] = np.median(nnmodel[model_iter]['y_train_pred_error'])
    nnmodel[model_iter]['test_medape'] = np.median(nnmodel[model_iter]['y_test_pred_error'])
    
    print('Neural Network Iteration {0}'.format(model_iter))
    print('Training Set: RMSE {0}, MedAE {1}, Explained Variance {2}, MedAPE {3}'.format(nnmodel[model_iter]['train_rmse'], nnmodel[model_iter]['train_medae'], nnmodel[model_iter]['train_explained_var'], nnmodel[model_iter]['train_medape']))
    print('Testing Set: RMSE {0}, MedAE {1}, Explained Variance {2}, MedAPE {3}'.format(nnmodel[model_iter]['test_rmse'], nnmodel[model_iter]['test_medae'], nnmodel[model_iter]['test_explained_var'], nnmodel[model_iter]['test_medape']))
    print(model.summary())
    model_iter = model_iter + 1
    # remove the output layer from previously built model
    model.pop()


  save_learned_models(nnmodel, algo, "nnmodel")
  print('Final Model--->')
  model1 = tf.keras.models.load_model(drive_location+algo+"_nnmodel.h5")
  model1.summary()


In [ ]:
def build_autogluon(algo):

  from tpot import TPOTRegressor

  dataset = load_learned_models(algo, "dataset")
  
  reg = TPOTRegressor(generations=5, population_size=50, verbosity=2, random_state=42)
  # Feed the tensorflow Dataset to the regressor.
  reg.fit(dataset['X_train_standardized'], dataset['y_train_standardized'])
  # Predict with the best model.
  
  y_train_pred = reg.predict(dataset['X_train_standardized'])
  y_test_pred = reg.predict(dataset['X_test_standardized'])

  autotpot = {}
  autotpot['y_train'] = dataset['y_train_standardized']
  autotpot['y_train_pred'] = y_train_pred
  autotpot['y_test'] = dataset['y_test_standardized']
  autotpot['y_test_pred'] = y_test_pred
  autotpot['train_rmse'] = mean_squared_error(dataset['y_train_standardized'],y_train_pred,squared=False)
  autotpot['train_medae'] = median_absolute_error(dataset['y_train_standardized'],y_train_pred)
  autotpot['train_explained_var'] = explained_variance_score(dataset['y_train_standardized'],y_train_pred)
  autotpot['test_rmse'] = mean_squared_error(dataset['y_test_standardized'],y_test_pred,squared=False)
  autotpot['test_medae'] = median_absolute_error(dataset['y_test_standardized'],y_test_pred)
  autotpot['test_explained_var'] = explained_variance_score(dataset['y_test_standardized'],y_test_pred)
  autotpot['y_train_pred_error'] = np.abs(dataset['y_train_standardized'] - y_train_pred/dataset['y_train_standardized']*100)
  autotpot['y_test_pred_error'] = np.abs(dataset['y_test_standardized'] - y_test_pred/dataset['y_test_standardized']*100)
  autotpot['train_medape'] = np.median(autotpot['y_train_pred_error'])
  autotpot['test_medape'] = np.median(autotpot['y_test_pred_error'])
    
  print('Auto Keras-->')
  print('Training Set: RMSE {0}, MedAE {1}, Explained Variance {2}, MedAPE {3}'.format(autotpot['train_rmse'], autotpot['train_medae'], autotpot['train_explained_var'], autotpot['train_medape']))
  print('Testing Set: RMSE {0}, MedAE {1}, Explained Variance {2}, MedAPE {3}'.format(autotpot['test_rmse'], autotpot['test_medae'], autotpot['test_explained_var'], autotpot['test_medape']))

  # Export as a Keras Model.
 # model = reg.export_model()
  #try:
   # model.save(drive_location+algo+"_model_autogluon", save_format="tf")
    #loaded_model = load_model(drive_location+algo+"_model_autogluon", custom_objects=ak.CUSTOM_OBJECTS)
    #loaded_model = load_model(drive_location+algo+"_model_autogluon")
    #print(loaded_model.predict(tf.expand_dims(dataset['X_test_standardized'], -1)))
  #except Exception:
   # model.save(drive_location+algo+"_autogluon.h5")

  save_learned_models(autotpot, algo, "autogluon")

In [ ]:
def set_pub():
    mpl.rcdefaults()
    mpl.rcParams['font.size'] = 14
    #mpl.rcParams['font.weight'] = 'bold'
    mpl.rcParams['xtick.labelsize'] = 'xx-large'
    mpl.rcParams['ytick.labelsize'] = 'xx-large'
    mpl.rcParams['figure.titlesize'] = 'xx-large'
    mpl.rcParams['legend.fontsize'] = 'xx-large'
    mpl.rcParams['axes.titlesize'] = 'xx-large'
    mpl.rcParams['axes.titleweight'] = 'bold'
    mpl.rcParams['axes.labelsize'] = 'xx-large'
    mpl.rcParams['axes.labelweight'] = 'bold'
    mpl.rcParams['lines.linewidth'] = 3
    mpl.rcParams['lines.markersize'] = 12
    print(mpl.rcParams['figure.titlesize'], mpl.rcParams['legend.fontsize'])
    plt.rc('figure', dpi=500)

In [ ]:
def capture_metric_in_csv(benchmark_title, algo_list):
  # First capture data in csv file for all algo
  metric_list = []
  for algo in algo_list:
    nnmodel = load_learned_models(algo, "nnmodel")
    final_nnmodel_key = list(nnmodel.keys())[-1]
    autotpot = load_learned_models(algo, "autogluon")
    metric_dict = {}
    metric_dict['algo'] = algo
    # RMSE
    metric_dict['nnmodel_train_rmse'] = nnmodel[final_nnmodel_key]['train_rmse']
    metric_dict['autotpot_train_rmse'] = autotpot['train_rmse']
    metric_dict['nnmodel_test_rmse'] = nnmodel[final_nnmodel_key]['test_rmse']
    metric_dict['autotpot_test_rmse'] = autotpot['test_rmse']
    # MedAE
    metric_dict['nnmodel_train_medae'] = nnmodel[final_nnmodel_key]['train_medae']
    metric_dict['autotpot_train_medae'] = autotpot['train_medae']
    metric_dict['nnmodel_test_medae'] = nnmodel[final_nnmodel_key]['test_medae']
    metric_dict['autotpot_test_medae'] = autotpot['test_medae']
    # MedAPE
    metric_dict['nnmodel_train_medape'] = nnmodel[final_nnmodel_key]['train_medape']
    metric_dict['autotpot_train_medape'] = autotpot['train_medape']
    metric_dict['nnmodel_test_medape'] = nnmodel[final_nnmodel_key]['test_medape']
    metric_dict['autotpot_test_medape'] = autotpot['test_medape']

    print(metric_dict)
    metric_list.append(metric_dict)
  metric_df = pd.DataFrame.from_dict(metric_list)
  metric_df.to_csv(drive_location+benchmark_title+"_metric.csv", line_terminator='\n', index=False)

In [ ]:
def plot_nnmodel_vs_autogluon(benchmark_title, algo_list):
  capture_metric_in_csv(benchmark_title, algo_list)

  metric_df = pd.read_csv(drive_location+benchmark_title+"_metric.csv")
  y_pos = np.arange(metric_df.shape[0])
  print(y_pos)


  # Plot Train and Test MedAE for nnmodel (our model) and autogluon model 
  colors = ['r','k','b','y']
  fig, axes = plt.subplots(figsize=(len(y_pos)*1.5, 4))
  rect1 = []
  rect2 = []
  rect3 = []
  rect4 = []
  width = 0.2
  rect1 = plt.bar(y_pos, metric_df['nnmodel_train_medae'], width, color=colors[0])
  rect2 = plt.bar(y_pos+width, metric_df['autotpot_train_medae'], width, color=colors[1])
  rect3 = plt.bar(y_pos+2*width, metric_df['nnmodel_test_medae'], width, color=colors[2])
  rect4 = plt.bar(y_pos+3*width, metric_df['autotpot_test_medae'], width, color=colors[3])
  
  plt.xticks(y_pos+1.5*width, metric_df['algo'], rotation=20, ha='right')
  plt.ylabel('MedAE')
  plt.xlabel(benchmark_title)
  #lgd = plt.legend(['Simulator Runtime (Scaled)', 'Physical System Runtime', 'Cross Prediction Runtime (Scaled)'], bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
  lgd = plt.legend((rect1, rect2, rect3,rect4), ("Our Model Train MedAE", "autotpot Model Train MedAE", "Our Model Test MedAE", "autotpot Test MedAE"), bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
  plt.show()
  strFile = drive_location + benchmark_title + "_nnmodel_vs_autotpot_medae"+plot_file_ext
  if os.path.isfile(strFile):
    os.remove(strFile)
  #fig.savefig(strFile, bbox_extra_artists=(lgd,), bbox_inches='tight')
  fig.savefig(strFile, bbox_inches='tight')


  # # Plot Train and Test MedAPE for nnmodel (our model) and autogluon model 
  # colors = ['r','k','b','y']
  # fig, axes = plt.subplots(figsize=(len(y_pos)*1.5, 4))
  # rect1 = []
  # rect2 = []
  # rect3 = []
  # rect4 = []
  # width = 0.2
  # rect1 = plt.bar(y_pos, metric_df['nnmodel_train_medape'], width, color=colors[0])
  # rect2 = plt.bar(y_pos+width, metric_df['autotpot_train_medape'], width, color=colors[1])
  # rect3 = plt.bar(y_pos+2*width, metric_df['nnmodel_test_medape'], width, color=colors[2])
  # rect4 = plt.bar(y_pos+3*width, metric_df['autotpot_test_medape'], width, color=colors[3])
  
  # plt.xticks(y_pos+1.5*width, metric_df['algo'], rotation=20, ha='right')
  # plt.ylabel('MedAPE')
  # plt.xlabel(benchmark_title)
  # #lgd = plt.legend(['Simulator Runtime (Scaled)', 'Physical System Runtime', 'Cross Prediction Runtime (Scaled)'], bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
  # lgd = plt.legend((rect1, rect2, rect3,rect4), ("Our Model Train MedAPE", "autogluon Model Train MedAPE", "Our Model Test MedAPE", "autogluon Test MedAPE"), bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
  # plt.show()
  # strFile = drive_location + benchmark_title + "_nnmodel_vs_autotpot_medape"+plot_file_ext
  # if os.path.isfile(strFile):
  #   os.remove(strFile)
  # #fig.savefig(strFile, bbox_extra_artists=(lgd,), bbox_inches='tight')
  # fig.savefig(strFile, bbox_inches='tight')


  # drop RMSE values for SPEC2006 Int Benchmarks due to very high value
  if (benchmark_title == 'SPEC_Int'):
    indexNames = metric_df[metric_df['algo'].isin(['401.bzip2', '456.hmmer', '458.sjeng', '462.libquantum', '473.astar'])].index
    #print(indexNames)
    metric_df = metric_df.drop(indexNames)
    #print(metric_df)
  y_pos = np.arange(metric_df.shape[0])
  print(y_pos)

   # Plot Train and Test RMSE for nnmodel (our model) and autogluon model 
  colors = ['r','k','b','y']
  fig, axes = plt.subplots(figsize=(len(y_pos)*1.5, 4))
  rect1 = []
  rect2 = []
  rect3 = []
  rect4 = []
  width = 0.2
  rect1 = plt.bar(y_pos, metric_df['nnmodel_train_rmse'], width, color=colors[0])
  rect2 = plt.bar(y_pos+width, metric_df['autotpot_train_rmse'], width, color=colors[1])
  rect3 = plt.bar(y_pos+2*width, metric_df['nnmodel_test_rmse'], width, color=colors[2])
  rect4 = plt.bar(y_pos+3*width, metric_df['autotpot_test_rmse'], width, color=colors[3])
  
  plt.xticks(y_pos+1.5*width, metric_df['algo'], rotation=20, ha='right')
  plt.ylabel('RMSE')
  plt.xlabel(benchmark_title)
  #lgd = plt.legend(['Simulator Runtime (Scaled)', 'Physical System Runtime', 'Cross Prediction Runtime (Scaled)'], bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
  lgd = plt.legend((rect1, rect2, rect3,rect4), ("Our Model Train RMSE", "autotpot Model Train RMSE", "Our Model Test RMSE", "autotpot Test RMSE"), bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
  plt.show()
  strFile = drive_location + benchmark_title + "_nnmodel_vs_autotpot_rmse"+plot_file_ext
  if os.path.isfile(strFile):
    os.remove(strFile)
  #fig.savefig(strFile, bbox_extra_artists=(lgd,), bbox_inches='tight')
  fig.savefig(strFile, bbox_inches='tight')


In [ ]:
def capture_iteration_metric_in_csv(benchmark_title, algo_list):
  # First capture data in csv file for all algo
  metric_list_train_rmse = []
  metric_list_train_medae = []
  metric_list_train_medape = []
  metric_list_test_rmse = []
  metric_list_test_medae = []
  metric_list_test_medape = []
  for algo in algo_list:
    nnmodel = load_learned_models(algo, "nnmodel")
    nnmodel_keys = list(nnmodel.keys())
    print(nnmodel_keys)
    metric_dict_train_rmse = {}
    metric_dict_train_medae = {}
    metric_dict_train_medape = {}
    metric_dict_test_rmse = {}
    metric_dict_test_medae = {}
    metric_dict_test_medape = {}

    metric_dict_train_rmse['algo'] = algo
    metric_dict_train_medae['algo'] = algo
    metric_dict_train_medape['algo'] = algo
    metric_dict_test_rmse['algo'] = algo
    metric_dict_test_medae['algo'] = algo
    metric_dict_test_medape['algo'] = algo
    
    for key in nnmodel_keys:
      # RMSE
      metric_dict_train_rmse[key] = nnmodel[key]['train_rmse']
      metric_dict_test_rmse[key] = nnmodel[key]['test_rmse']
      # MedAE
      metric_dict_train_medae[key] = nnmodel[key]['train_medae']
      metric_dict_test_medae[key] = nnmodel[key]['test_medae']
      # MedAPE
      metric_dict_train_medape[key] = nnmodel[key]['train_medape']
      metric_dict_test_medape[key] = nnmodel[key]['test_medape']

    print(metric_dict_train_rmse)
    metric_list_train_rmse.append(metric_dict_train_rmse)
    metric_df_train_rmse = pd.DataFrame.from_dict(metric_list_train_rmse)
    metric_df_train_rmse.to_csv(drive_location+benchmark_title+"_metric_train_rmse.csv", line_terminator='\n', index=False)

    metric_list_test_rmse.append(metric_dict_test_rmse)
    metric_df_test_rmse = pd.DataFrame.from_dict(metric_list_test_rmse)
    metric_df_test_rmse.to_csv(drive_location+benchmark_title+"_metric_test_rmse.csv", line_terminator='\n', index=False)

    metric_list_train_medae.append(metric_dict_train_medae)
    metric_df_train_medae = pd.DataFrame.from_dict(metric_list_train_medae)
    metric_df_train_medae.to_csv(drive_location+benchmark_title+"_metric_train_medae.csv", line_terminator='\n', index=False)

    metric_list_test_medae.append(metric_dict_test_medae)
    metric_df_test_medae = pd.DataFrame.from_dict(metric_list_test_medae)
    metric_df_test_medae.to_csv(drive_location+benchmark_title+"_metric_test_medae.csv", line_terminator='\n', index=False)

    metric_list_train_medape.append(metric_dict_train_medape)
    metric_df_train_medape = pd.DataFrame.from_dict(metric_list_train_medape)
    metric_df_train_medape.to_csv(drive_location+benchmark_title+"_metric_train_medape.csv", line_terminator='\n', index=False)

    metric_list_test_medape.append(metric_dict_test_medape)
    metric_df_test_medape = pd.DataFrame.from_dict(metric_list_test_medape)
    metric_df_test_medape.to_csv(drive_location+benchmark_title+"_metric_test_medape.csv", line_terminator='\n', index=False) 

In [ ]:
def plot_iteration_metric(benchmark_title, algo_list):
  capture_iteration_metric_in_csv(benchmark_title, algo_list)

  metrics = ['train_rmse','test_rmse','train_medae','test_medae','train_medape','test_medape']

  for metric in metrics:
    metric_df = pd.read_csv(drive_location+benchmark_title+"_metric_"+metric+".csv")

    fig, axes = plt.subplots(1,1)
    for algo_idx, algo in enumerate(metric_df['algo']):
      print(metric_df.values[algo_idx][1:])
      x_pos = np.arange(len(metric_df.values[algo_idx][1:]))
      print(x_pos)
      plt.plot(x_pos, metric_df.values[algo_idx][1:])
    plt.xticks(x_pos)
    plt.ylabel(metric.upper())
    plt.xlabel('Iterations')
    plt.title('Iterations vs '+metric.upper())
    ldg = plt.legend(metric_df['algo'], bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show()
    strFile = drive_location + benchmark_title + "_metric_"+metric+plot_file_ext
    if os.path.isfile(strFile):
      os.remove(strFile)
    #fig.savefig(strFile, bbox_extra_artists=(lgd,), bbox_inches='tight')
    fig.savefig(strFile, bbox_inches='tight')

In [ ]:
import pandas as pd
!pip install category_encoders
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle
import gc
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, median_absolute_error, explained_variance_score
import numpy as np
import tensorflow as tf

!pip install tpot
#import autogluon as ak
from tensorflow.keras.models import load_model
drive_location = r"/content/drive/MyDrive/Colab Notebooks/"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 2.5 MB/s 


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 87 kB 1.5 MB/s 
     |████████████████████████████████| 160 kB 15.3 MB/s 
     |████████████████████████████████| 192.9 MB 48 kB/s 
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11956 sha256=cc99cff220a7dd82c844f01b456d44c28000cfe99d8b905174694e8b12e80b3b
  Stored in directory: /root/.cache/pip/wheels/e2/d2/79/eaf81edb391e27c87f51b8ef901ecc85a5363dc96b8b8d71e3
Successfully built stopit
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


In [ ]:
# Mantevo Suite
# benchmark = "Mantevo"
#algo="mantevo_miniFE"
#algo_fname="runtimes_final_mantevo_miniFE"
#NPB Suite
#benchmark = "NPB"
#algo = "npb_ep"
#algo_fname="runtimes_final_npb_ep"
benchmark = "NPB"
algo = "npb_mg"
algo_fname="runtimes_final_npb_mg"
#SPEC 2006 Float
#benchmark = "SPEC2006"
#algo = "leslie3d"
# algo_fname = "437.leslie3d"
#SPEC 2006 Int
#benchmark = "SPEC2006"
#algo = "perlbench"
#algo_fname = "400.perlbench"

spec_float_benchmark_list = ["SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2006","SPEC2006","SPEC2006","SPEC2006","SPEC2006","SPEC2006","SPEC2006","SPEC2006","SPEC2006"]
spec_float_algo_list = ["603.bwaves_s","607.cactuBSSN_s","619.lbm_s","621.wrf_s","627.cam4_s","628.pop2_s","638.imagick_s","644.nab_s","649.fotonik3d_s","654.roms_s","416.gamess","433.milc","434.zeusmp","435.gromacs","437.leslie3d","447.dealII","450.soplex","453.povray","454.calculix"]
spec_float_algo_fname_list = ["603.bwaves_s","607.cactuBSSN_s","619.lbm_s","621.wrf_s","627.cam4_s","628.pop2_s","638.imagick_s","644.nab_s","649.fotonik3d_s","654.roms_s","416.gamess","433.milc","434.zeusmp","435.gromacs","437.leslie3d","447.dealII","450.soplex","453.povray","454.calculix"]

spec_int_benchmark_list = ["SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2006","SPEC2006","SPEC2006","SPEC2006","SPEC2006"]
spec_int_algo_list = ["600.perlbench_s","602.gcc_s","605.mcf_s","620.omnetpp_s","623.xalancbmk_s","625.x264_s","631.deepsjeng_s","641.leela_s","648.exchange2_s","657.xz_s","401.bzip2","456.hmmer","458.sjeng","462.libquantum","473.astar"]
spec_int_algo_fname_list = ["600.perlbench_s","602.gcc_s","605.mcf_s","620.omnetpp_s","623.xalancbmk_s","625.x264_s","631.deepsjeng_s","641.leela_s","648.exchange2_s","657.xz_s","401.bzip2","456.hmmer","458.sjeng","462.libquantum","473.astar"]

npb_other_benchmark_list = ["NPB","NPB","NPB","NPB","OTHER","OTHER","OTHER"]
npb_other_algo_list = ["npb_ep","npb_mg","npb_sp","npb_sp-mz","matmul","montecarlo","quicksort"]
npb_other_algo_fname_list = ["npb_ep","npb_mg","npb_sp","npb_sp-mz","matmul","montecarlo","quicksort"]


# Assign variable with which list is to be processed
benchmark_list = spec_int_benchmark_list
algo_list = spec_int_algo_list
algo_fname_list = spec_int_algo_fname_list
for algo_idx, algo in enumerate(algo_list):
  #Perform train-test data split and scaling
  perform_dataset_ops(benchmark_list[algo_idx], algo_list[algo_idx], algo_fname_list[algo_idx])


  build_nn_model(algo)

  build_autogluon(algo)



SPEC2017 600.perlbench_s 600.perlbench_s
df_orig====================>                                             system_name  \
0     ASUS RS700-E9(Z11PP-D24) Server System (2.70 G...   
1     ASUS RS700-E9(Z11PP-D24) Server System (2.10 G...   
2     ASUS WS C621E SAGE Server System (2.50 GHz- In...   
3     ASUS WS C621E SAGE Server System (3.00 GHz- In...   
4     ASUS RS720Q-E9(Z11PH-D12) Server System (2.10 ...   
...                                                 ...   
1981  ProLiant DL360 Gen10 (1.70 GHz- Intel Xeon Bro...   
1982  ProLiant DL360 Gen10 (3.50 GHz- Intel Xeon Gol...   
1983  ProLiant DL380 Gen10 (2.20 GHz- Intel Xeon Gol...   
1984  Synergy 480 Gen10 (2.00 GHz- Intel Xeon Gold 6...   
1985  ProLiant ML110 Gen10 (3.60 GHz- Intel Xeon Gol...   

                          arch  cpu_clock     isa  ld_shared_by_cores  \
0         Intel Xeon Gold 6150        2.7  X86_64                   1   
1     Intel Xeon Platinum 8176        2.1  X86_64                   1   
2 

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Optimization Progress:   0%|          | 0/300 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: -0.1632587859158095

Generation 2 - Current best internal CV score: -0.1632587859158095

Generation 3 - Current best internal CV score: -0.15038881840971252

Generation 4 - Current best internal CV score: -0.15038881840971252

Generation 5 - Current best internal CV score: -0.15038881840971252

Best pipeline: XGBRegressor(input_matrix, learning_rate=0.5, max_depth=6, min_child_weight=5, n_estimators=100, n_jobs=1, objective=reg:squarederror, subsample=1.0, verbosity=0)
Auto Keras-->
Training Set: RMSE 0.2195570840235006, MedAE 0.03595127062604492, Explained Variance 0.9517946872406985, MedAPE 96.3597418974998
Testing Set: RMSE 0.32579217287519435, MedAE 0.053284923572713305, Explained Variance 0.9054753808365835, MedAPE 95.93820148549979
SPEC2017 602.gcc_s 602.gcc_s
df_orig====================>                                             system_name  \
0     ASUS RS700-E9(Z11PP-D24) Server System (2.70 G...   
1     ASUS RS700-E9(Z11PP-D2

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

Optimization Progress:   0%|          | 0/300 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: -0.021440467060322703

Generation 2 - Current best internal CV score: -0.021440467060322703

Generation 3 - Current best internal CV score: -0.021336954025593196

Generation 4 - Current best internal CV score: -0.019339395863314946

Generation 5 - Current best internal CV score: -0.019339395863314946

Best pipeline: XGBRegressor(OneHotEncoder(AdaBoostRegressor(input_matrix, learning_rate=0.01, loss=linear, n_estimators=100), minimum_fraction=0.05, sparse=False, threshold=10), learning_rate=1.0, max_depth=8, min_child_weight=20, n_estimators=100, n_jobs=1, objective=reg:squarederror, subsample=1.0, verbosity=0)
Auto Keras-->
Training Set: RMSE 0.07696386123095479, MedAE 0.013965707777445768, Explained Variance 0.9940765640706775, MedAPE 100.12489130802456
Testing Set: RMSE 0.23923834751170972, MedAE 0.025363776076719025, Explained Variance 0.937558499590139, MedAPE 98.30820526600233
SPEC2006 456.hmmer 456.hmmer
df_orig====================>

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Optimization Progress:   0%|          | 0/300 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: -0.03884599977119976


In [ ]:
#Plotting

import matplotlib as mpl
from matplotlib import pyplot as plt
import os
drive_location = r"/content/drive/MyDrive/Colab Notebooks/"

In [ ]:
#Plotting

#benchmark_list = ["NPB", "NPB", "SPEC2006", "SPEC2006"]
#algo_list = ["npb_ep", "npb_mg", "leslie3d", "perlbench"]

spec_float_benchmark_list = ["SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2006","SPEC2006","SPEC2006","SPEC2006","SPEC2006","SPEC2006","SPEC2006","SPEC2006","SPEC2006"]
spec_float_algo_list = ["603.bwaves_s","607.cactuBSSN_s","619.lbm_s","621.wrf_s","627.cam4_s","628.pop2_s","638.imagick_s","644.nab_s","649.fotonik3d_s","654.roms_s","416.gamess","433.milc","434.zeusmp","435.gromacs","437.leslie3d","447.dealII","450.soplex","453.povray","454.calculix"]
spec_float_algo_fname_list = ["603.bwaves_s","607.cactuBSSN_s","619.lbm_s","621.wrf_s","627.cam4_s","628.pop2_s","638.imagick_s","644.nab_s","649.fotonik3d_s","654.roms_s","416.gamess","433.milc","434.zeusmp","435.gromacs","437.leslie3d","447.dealII","450.soplex","453.povray","454.calculix"]
spec_float_benchmark_title = 'SPEC_Float'

spec_int_benchmark_list = ["SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2017","SPEC2006","SPEC2006","SPEC2006","SPEC2006","SPEC2006"]
spec_int_algo_list = ["600.perlbench_s","602.gcc_s","605.mcf_s","620.omnetpp_s","623.xalancbmk_s","625.x264_s","631.deepsjeng_s","641.leela_s","648.exchange2_s","657.xz_s","401.bzip2","456.hmmer","458.sjeng","462.libquantum","473.astar"]
spec_int_algo_fname_list = ["600.perlbench_s","602.gcc_s","605.mcf_s","620.omnetpp_s","623.xalancbmk_s","625.x264_s","631.deepsjeng_s","641.leela_s","648.exchange2_s","657.xz_s","401.bzip2","456.hmmer","458.sjeng","462.libquantum","473.astar"]
spec_int_benchmark_title = 'SPEC_Int'

npb_other_benchmark_list = ["NPB","NPB","NPB","NPB","OTHER","OTHER","OTHER"]
npb_other_algo_list = ["npb_ep","npb_mg","npb_sp","npb_sp-mz","matmul","montecarlo","quicksort"]
npb_other_algo_fname_list = ["npb_ep","npb_mg","npb_sp","npb_sp-mz","matmul","montecarlo","quicksort"]
npb_other_benchmark_title = 'NPB_OTHER'

# Assign variable with which list is to be processed
benchmark_list = spec_int_benchmark_list
algo_list = spec_int_algo_list
algo_fname_list = spec_int_algo_fname_list
benchmark_title = spec_int_benchmark_title
plot_file_ext=".eps"

set_pub()
plot_nnmodel_vs_autogluon(benchmark_title, algo_list)

plot_iteration_metric(benchmark_title, algo_list)